In [1]:
!pip install --upgrade --force-reinstall "protobuf==4.21.12"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 8.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.21.12 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 4.21.12 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.21.12 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [2]:
# ===================================================================
# PASSO 0: PREPARAR O AMBIENTE
# Clonar o repositório para ter acesso ao código-fonte
# ===================================================================

import google.protobuf
import pkg_resources

!git clone https://github.com/unclecode/crawl4ai.git
# Movemos para o diretório correto para que os arquivos sejam encontrados
%cd crawl4ai/crawl4ai 

# 1. Instalar as bibliotecas necessárias
!pip install transformers accelerate bitsandbytes

# 2. Importar tudo
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import os
import json
from IPython.display import display, Markdown
import torch
torch.cuda.empty_cache()

# 3. Configurar a quantização (para caber na GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 4. Carregar o Modelo e o Tokenizer
# O ideal é adicionar o modelo como um "Kaggle Dataset" para ser mais rápido.
# Se não, ele baixará do Hugging Face.
model_id = "/kaggle/input/codellama/hf/7b-instruct" 

if not os.path.exists(model_id):
    print(f"Modelo não encontrado em {model_id}, baixando do Hub...")
    model_id = "codellama/CodeLlama-7b-Instruct-hf"

print(f"Carregando modelo de: {model_id}")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "codellama/CodeLlama-7b-Instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to("cuda:0")

# 5. Funções "Helper" essenciais
def ask_codellama(prompt_text, max_new_tokens=2048):
    """
    Função para enviar um prompt formatado para o CodeLlama e receber uma resposta.
    """
    prompt_template = f"<s>[INST] {prompt_text} [/INST]"
    model.to("cuda")
    inputs = tokenizer(prompt_template, return_tensors="pt").to("cuda")
    
    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    clean_response = response_text.split("[/INST]")[-1].strip()
    
    return clean_response

def get_skeleton(file_path):
    """
    Lê um arquivo Python e extrai apenas as linhas de import, class, def e decoradores.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        skeleton_lines = re.findall(r"^\s*(?:import|from|class|def|@).*", content, re.MULTILINE)
        return "\n".join(skeleton_lines)
    except FileNotFoundError:
        # A mensagem de erro agora mostrará o diretório atual para ajudar no debug
        return f"Erro: Arquivo {file_path} não encontrado no diretório atual: {os.getcwd()}."
    except Exception as e:
        return f"Erro ao ler {file_path}: {e}"

# O arquivo de saída será salvo um nível acima, no diretório /kaggle/working/
EVIDENCE_FILE_PATH = '/kaggle/working/architectural_evidence.json'

def append_evidence_to_file(arquivo, padrao, evidencia):
    """
    Carrega um arquivo JSON, adiciona uma nova evidência e o salva de volta.
    """
    try:
        if os.path.exists(EVIDENCE_FILE_PATH):
            with open(EVIDENCE_FILE_PATH, 'r', encoding='utf-8') as f:
                data = json.load(f)
        else:
            data = {}
    except (json.JSONDecodeError, FileNotFoundError):
        data = {}

    if arquivo not in data:
        data[arquivo] = []

    entry = {"padrao": padrao, "evidencia": evidencia}
    if entry not in data[arquivo]:
        data[arquivo].append(entry)
        print(f"[Feedback do Script] Evidência salva para '{arquivo}': Padrão='{padrao}'")

    with open(EVIDENCE_FILE_PATH, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

/tmp/ipykernel_48/1387842953.py:7: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


Cloning into 'crawl4ai'...
remote: Enumerating objects: 10109, done.
remote: Counting objects: 100% (1885/1885), done.
remote: Compressing objects: 100% (465/465), done.
remote: Total 10109 (delta 1668), reused 1421 (delta 1420), pack-reused 8224 (from 3)
Receiving objects: 100% (10109/10109), 144.23 MiB | 34.96 MiB/s, done.
Resolving deltas: 100% (6444/6444), done.
/kaggle/working/crawl4ai/crawl4ai
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 M

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
# --- INÍCIO DA LÓGICA DE INVESTIGAÇÃO ---

FILES_TO_INVESTIGATE = ["async_webcrawler.py", "model_loader.py", "browser_manager.py", "async_dispatcher.py"]

for file_to_investigate in FILES_TO_INVESTIGATE:
    print(f"\n--- Iniciando Investigação de Fato-Interpretação: {file_to_investigate} ---")
    skeleton_code = get_skeleton(file_to_investigate)
    if "Erro:" in skeleton_code:
        print(skeleton_code)
        continue

    # --- PROMPT DE FATOS COMPLETO (RESTAURADO) ---
    prompt_fact_finding = f"""
    Eu sou um analista de software. NÃO me diga quais padrões de projeto você vê.
    Apenas responda 'Sim' ou 'Não' para os seguintes fatos sobre o esqueleto de código abaixo.

    Esqueleto do Código:
    ---
    {skeleton_code}
    ---

    Responda em português. Use este formato exato:
    FATO 1 (Strategy): O código define uma classe base abstrata (ABC) e outras classes que herdam dela?
    RESPOSTA: [Sim/Não], [Evidência: ...nome da classe base...]

    FATO 2 (Factory Method): O código define métodos estáticos (ex: 'from_string' ou 'from_dict') que criam e retornam uma instância da própria classe?
    RESPOSTA: [Sim/Não], [Evidência: ...nome do método...]

    FATO 3 (Singleton): O código tem lógica para garantir que apenas uma instância de uma classe seja criada? (Ex: verificar se 'self._instance' é None)?
    RESPOSTA: [Sim/Não]

    FATO 4 (Builder): O código define uma classe com múltiplos métodos (ex: '.set_option(..)') que retornam 'self' para permitir encadeamento?
    RESPOSTA: [Sim/Não], [Evidência: ...nome do método...]

    FATO 5 (Decorator): O código define uma função que recebe outra função como argumento e retorna uma nova função 'wrapper'?
    RESPOSTA: [Sim/Não]

    FATO 6 (Adapter): O código define uma classe que, em seu construtor ('__init__'), armazena uma instância de outro objeto (ex: 'self.adaptado = objeto') para mudar sua interface?
    RESPOSTA: [Sim/Não]

    FATO 7 (Proxy): O código define uma classe que herda de uma ABC, mas em seu '__init__' ela armazena uma instância de *outra* classe que *também* herda daquela ABC?
    RESPOSTA: [Sim/Não]

    FATO 8 (Composite): O código define uma classe que tem uma lista de objetos *do seu próprio tipo*? (Ex: `class Node: self.children: List[Node]`).
    RESPOSTA: [Sim/Não]

    FATO 9 (Observer): O código define métodos como 'add_observer' ou 'notify_observers'?
    RESPOSTA: [Sim/Não]

    FATO 10 (Template Method): O código define uma classe base (ABC) com um método *concreto* (não-abstrato) que chama outros métodos *abstratos*?
    RESPOSTA: [Sim/Não], [Evidência: ...nome do método concreto...]

    FATO 11 (Command): O código define uma classe base (ABC) com um único método abstrato (ex: 'execute()')?
    RESPOSTA: [Sim/Não]

    FATO 12 (Cache): O código usa o decorador '@lru_cache' ou '@cache'?
    RESPOSTA: [Sim/Não]

    FATO 13 (Semaphore): O código importa ou usa 'asyncio.Semaphore' ou 'threading.Semaphore'?
    RESPOSTA: [Sim/Não]
    """
    print(f"--- Pergunta ao CodeLlama (Passo A): Buscando 13 FATOS em {file_to_investigate} ---")
    analysis_facts = ask_codellama(prompt_fact_finding)
    display(Markdown(f"**Relatório de Fatos do CodeLlama:**\n```\n{analysis_facts}\n```"))

    prompt_interpretation = f"""
    Agora, aja como um Arquiteto de Software.
    Aqui está o Relatório de Fatos sobre o arquivo '{file_to_investigate}':
    ---
    {analysis_facts}
    ---
    Baseado *apenas* nos fatos com resposta 'Sim', liste os Padrões de Projeto que foram confirmados.
    Se a resposta foi 'Não' para um fato, ignore esse padrão.
    Use este formato exato para *cada* padrão que você confirmar:

    BLOCO_PADRAO
    PADRAO: [Nome do Padrão (ex: Strategy, Factory Method)]
    EVIDENCIA: [A evidência que o Relatório de Fatos forneceu]
    FIM_BLOCO
    """
    print(f"--- Pergunta ao CodeLlama (Passo B): Interpretando os FATOS ---")
    summary = ask_codellama(prompt_interpretation)
    display(Markdown(f"**Interpretação do CodeLlama:**\n```\n{summary}\n```"))

    try:
        pattern_blocks = re.findall(r"BLOCO_PADRAO(.*?)FIM_BLOCO", summary, re.DOTALL | re.IGNORECASE)
        if not pattern_blocks:
            print("[Feedback do Script] Nenhum padrão foi confirmado pelo CodeLlama para este arquivo.")
        else:
            print(f"[Feedback do Script] Encontrados {len(pattern_blocks)} padrões confirmados. Processando e salvando...")
            for block in pattern_blocks:
                padrao_match = re.search(r"PADRAO:\s*(.*)", block, re.IGNORECASE)
                evidencia_match = re.search(r"EVIDENCIA:\s*(.*)", block, re.IGNORECASE)
                if padrao_match and evidencia_match:
                    padrao = padrao_match.group(1).strip()
                    evidencia = evidencia_match.group(1).strip()
                    append_evidence_to_file(arquivo=file_to_investigate, padrao=padrao, evidencia=evidencia)
                else:
                    print(f"[Feedback do Script] Bloco ignorado (não conseguiu parsear): {block}")
    except Exception as e:
        print(f"ERRO ao parsear o resumo do CodeLlama: {e}")
        append_evidence_to_file(arquivo=file_to_investigate, padrao="Erro na Análise", evidencia=str(e))

    print(f"\n--- Investigação de {file_to_investigate} concluída. Resultado salvo em '{EVIDENCE_FILE_PATH}' ---")

print("\n--- Todas as investigações foram concluídas! ---")


--- Iniciando Investigação de Fato-Interpretação: async_webcrawler.py ---
--- Pergunta ao CodeLlama (Passo A): Buscando 13 FATOS em async_webcrawler.py ---


**Relatório de Fatos do CodeLlama:**
```
FATO 1 (Strategy): Não, não há evidência de uma classe base abstrata (ABC) sendo definida.

FATO 2 (Factory Method): Sim, o código define métodos estáticos 'from_string' e 'from_dict' que criam e retornam uma instância da própria classe.

FATO 3 (Singleton): Não, não há evidência de lógica para garantir que apenas uma instância da classe seja criada.

FATO 4 (Builder): Não, não há evidência de métodos como '.set_option(..)' que retornam 'self' para permitir encadeamento.

FATO 5 (Decorator): Sim, o código define uma função que recebe outra função como argumento e retorna uma nova função 'wrapper'.

FATO 6 (Adapter): Não, não há evidência de uma classe que armazena uma instância de outro objeto para mudar sua interface.

FATO 7 (Proxy): Não, não há evidência de uma classe que herda de uma ABC, mas em seu '__init__' armazena uma instância de outra classe que também herda daquela ABC.

FATO 8 (Composite): Não, não há evidência de uma classe que tem uma lista de objetos do seu próprio tipo.

FATO 9 (Observer): Não, não há evidência de métodos como 'add_observer' ou 'notify_observers'.

FATO 10 (Template Method): Não, não há evidência de uma classe base (ABC) com um método concreto (não-abstrato) que chama outros métodos abstratos.

FATO 11 (Command): Não, não há evidência de uma classe base (ABC) com um único método abstrato (ex: 'execute()').

FATO 12 (Cache): Não, não há evidência de o decorador '@lru_cache' ou '@cache' sendo usado.

FATO 13 (Semaphore): Não, não há evidência de 'asyncio.Semaphore' ou 'threading.Semaphore' serem importados ou usados.
```

--- Pergunta ao CodeLlama (Passo B): Interpretando os FATOS ---


**Interpretação do CodeLlama:**
```
BLOCO_PADRAO
PADRAO: Decorator
EVIDENCIA: O código define uma função que recebe outra função como argumento e retorna uma nova função 'wrapper'.
FIM_BLOCO
```

[Feedback do Script] Encontrados 1 padrões confirmados. Processando e salvando...
[Feedback do Script] Evidência salva para 'async_webcrawler.py': Padrão='Decorator'

--- Investigação de async_webcrawler.py concluída. Resultado salvo em '/kaggle/working/architectural_evidence.json' ---

--- Iniciando Investigação de Fato-Interpretação: model_loader.py ---
--- Pergunta ao CodeLlama (Passo A): Buscando 13 FATOS em model_loader.py ---


...

[Feedback do Script] Encontrados 1 padrões confirmados. Processando e salvando...
[Feedback do Script] Evidência salva para 'browser_manager.py': Padrão='Singleton'

--- Investigação de browser_manager.py concluída. Resultado salvo em '/kaggle/working/architectural_evidence.json' ---

--- Iniciando Investigação de Fato-Interpretação: async_dispatcher.py ---
--- Pergunta ao CodeLlama (Passo A): Buscando 13 FATOS em async_dispatcher.py ---


**Relatório de Fatos do CodeLlama:**
```
FATO 1 (Strategy): Não, não há evidência de uma classe base abstrata (ABC) no código.

FATO 2 (Factory Method): Sim, o código define métodos estáticos (ex: 'from_string' ou 'from_dict') que criam e retornam uma instância da própria classe.

FATO 3 (Singleton): Não, não há evidência de lógica para garantir que apenas uma instância de uma classe seja criada.

FATO 4 (Builder): Não, não há evidência de uma classe com múltiplos métodos (ex: '.set_option(..)') que retornam 'self' para permitir encadeamento.

FATO 5 (Decorator): Não, não há evidência de uma função que recebe outra função como argumento e retorna uma nova função 'wrapper'.

FATO 6 (Adapter): Não, não há evidência de uma classe que, em seu construtor ('__init__'), armazena uma instância de outro objeto para mudar sua interface.

FATO 7 (Proxy): Não, não há evidência de uma classe que herda de uma ABC, mas em seu '__init__' ela armazena uma instância de *outra* classe que *também* herda daquela ABC.

FATO 8 (Composite): Não, não há evidência de uma classe que tem uma lista de objetos *do seu próprio tipo* (ex: `class Node: self.children: List[Node]`).

FATO 9 (Observer): Não, não há evidência de métodos como 'add_observer' ou 'notify_observers'.

FATO 10 (Template Method): Não, não há evidência de uma classe base (ABC) com um método *concreto* (não-abstrato) que chama outros métodos *abstratos* (ex: 'execute()').

FATO 11 (Command): Não, não há evidência de uma classe base (ABC) com um único método abstrato (ex: 'execute()').

FATO 12 (Cache): Não, não há evidência de o decorador '@lru_cache' ou '@cache'.

FATO 13 (Semaphore): Sim, o código importa ou usa 'asyncio.Semaphore' ou 'threading.Semaphore'.
```

--- Pergunta ao CodeLlama (Passo B): Interpretando os FATOS ---


**Interpretação do CodeLlama:**
```
BLOCO_PADRAO
PADRAO: Semaphore
EVIDENCIA: O código importa ou usa 'asyncio.Semaphore' ou 'threading.Semaphore'.
FIM_BLOCO
```

[Feedback do Script] Encontrados 1 padrões confirmados. Processando e salvando...
[Feedback do Script] Evidência salva para 'async_dispatcher.py': Padrão='Semaphore'

--- Investigação de async_dispatcher.py concluída. Resultado salvo em '/kaggle/working/architectural_evidence.json' ---

--- Todas as investigações foram concluídas! ---


In [4]:
import json
import os
import re
import gc
import torch
from IPython.display import display, Markdown
# Usei a filtragem de issues "architecture" e "project patterns"
# Supondo que a função 'ask_codellama' já está definida no ambiente do seu notebook.

# ===================================================================
# DICA: Reinicie o kernel do notebook antes de executar esta célula
# para garantir que a memória da GPU esteja limpa.
# ===================================================================
print("Limpando a memória da GPU antes de começar...")
gc.collect()
torch.cuda.empty_cache()
print("Memória limpa. Iniciando o processo de análise.")


# ===================================================================
# FASE 1: DADOS DAS ISSUES (COLETADOS E EMBUTIDOS)
# ===================================================================
candidate_issues_data = [
    {'number': 1595, 'title': 'docs: Self-hosting guide with comprehensive monitoring documentation', 'body': '## Summary\n\nThis PR transforms the Docker deployment documentation...'},
    {'number': 1550, 'title': 'feat: Setup UV workspace with microservices architecture', 'body': '## Description\nInitial setup of UV workspace...'},
    {'number': 1572, 'title': 'Fix cdp setting with managed browser ', 'body': '# Summary\r\nThis PR fixes a concurrency bug...'},
    {'number': 1570, 'title': 'Add comprehensive company website scraper using Crawl4AI', 'body': 'This commit introduces a powerful Python script...'},
    {'number': 1556, 'title': 'Release/v0.7.6', 'body': 'Add Webhook Support for Docker Job Queue API...'},
    {'number': 1547, 'title': 'chore: update lxml version', 'body': '## Summary\r\nUpdates lxml dependency to 6.0...'},
    {'number': 1527, 'title': 'Release/v0.7.5', 'body': '# 🚀 Crawl4AI v0.7.5: The Docker Hooks & Security Update...'},
    {'number': 1519, 'title': 'feat(mcp): Migrate from SSE to HTTP transport and add configurable port', 'body': 'Closes: #1316\n\n## Problem\nMCP server suffered...'},
    {'number': 1514, 'title': 'Task 2', 'body': '## Summary...'},
    {'number': 1501, 'title': 'feat(StealthAdapter): fix stealth features for Playwright integration', 'body': '## Summary\r\nFixed stealth mode implementation...'},
    {'number': 1495, 'title': 'feat(ManagedBrowser): add viewport size configuration for browser launch', 'body': '### Summary\r\nThis change fixes an issue where the viewport settings...'},
    {'number': 1471, 'title': 'Fix: allow custom LLM providers for adaptive crawler embedding config…', 'body': '## Summary\r\n  - Change embedding_llm_config from Dict to Union...'},
    {'number': 1470, 'title': '#1268 fix: update redirected_url to current page URL and enhance normalize_url function', 'body': '## Summary\n\nFixes #1268 - URL resolution...'},
    {'number': 1465, 'title': 'docs: make example of using a Filter with `DefaultMarkdownGenerator` directly runnable', 'body': '## Summary\r\nUpdate the example of using a Filter...'},
    {'number': 1463, 'title': 'Add configurable device_scale_factor for improved screenshot quality', 'body': '## Context...'},
    {'number': 1436, 'title': 'fix(docker): resolve filter serialization and JSON encoding errors in deep crawl strategy', 'body': '### Summary...'},
    {'number': 1420, 'title': 'Feature: Add opt-in telemetry system to help improve Crawl4AI stability through anonymous crash reporting', 'body': '## Summary\r\nAdd opt-in telemetry system...'},
    {'number': 1417, 'title': 'feat(browser): Add CDP Headers Support for Remote Browser Integration  ', 'body': '## Summary\r\nThis update adds support for CDP headers...'},
    {'number': 1402, 'title': '(fix): Docker Streaming Crawl with Advanced Deep-Crawl Strategies', 'body': '## Summary\r\n#1052...'},
    {'number': 1399, 'title': 'Scraptest 1', 'body': '## Summary...'},
    {'number': 1310, 'title': '[Bug]: Documentation Mismatch and AttributeError in v0.7.0', 'body': '### crawl4ai version\n\n0.7.0...'},
    {'number': 1290, 'title': 'feat(extraction): support type list pipeline in JsonElementExtraction', 'body': '# Summary...This PR enhances JsonElementExtractionStrategy...'},
    {'number': 1274, 'title': 'Devcontainer support and integrated with vscode for agile development and testing', 'body': '## Summary\r\nAdded Development Container support...'},
    {'number': 1255, 'title': 'Fix JSONCSSSelector to Handle Sibling CSS Selectors', 'body': '## Summary...'},
    {'number': 1234, 'title': 'Fix TypeError when keep_data_attributes=False by ensuring list concat…', 'body': '\r\n##  Summary...'},
    {'number': 1212, 'title': 'Feat: added stateless streamable_http transport for MCP at /mcp/http', 'body': '## Summary\r\n\r\nadded stateless streamable_http endpoint...'},
    {'number': 1208, 'title': 'refactor(docker-api): Update Markdown request handling for multiple URLs and performance metrics', 'body': '## Summary\r\nThis pull request updates the Markdown endpoint...'},
    {'number': 1180, 'title': 'feat: Add CallbackURLFilter for custom URL filtering in deep crawling', 'body': '## Summary\r\nThis PR adds a new `CallbackURLFilter` class...'},
    {'number': 999, 'title': 'feat: adding filters that filter based on regular expressions of crawl4ai.deep_crawling.filters', 'body': '## Summary\r\nAdd filters that filter based on regular expressions...'},
    {'number': 680, 'title': 'docs(README): add installation steps and create new Jupyter notebook for Legat4me', 'body': '## Summary by CodeRabbit...'},
    {'number': 399, 'title': 'Error using crawler: maximum recursion depth exceeded', 'body': 'I\'m getting the `maximum recursion depth exceeded` error...'},
    {'number': 360, 'title': 'Passing arguments to hook to perform basic auth', 'body': "Hello! I am working on a solution where I use scrapy..."},
    {'number': 354, 'title': 'When running in docker container', 'body': "Hi,\r\n\r\nWhen I'm running crawl4ai in docker container..."},
    {'number': 330, 'title': 'Playwright is to support the intercept Request', 'body': 'Playwright is to support the intercept Request...'},
    {'number': 325, 'title': 'Code blocks lose formatting when converting from HTML to markdown', 'body': 'When converting HTML `<pre>` and `<code>` tags...'},
    {'number': 184, 'title': 'Returned markdown missing formatting and links', 'body': "Hi @unclecode - First of all thanks for your amazing work!..."},
    {'number': 110, 'title': 'Example on whole-blog crawling?', 'body': 'Thanks for creating alternatives to [FireCrawl](https://github.com/mendableai/firecrawl)...'},
    {'number': 105, 'title': 'Crawling error', 'body': "hey and thanks for this nice package!..."},
    {'number': 71, 'title': 'Support extraction of media files from <source> tag in HTML', 'body': '[Source](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/source) tags...'},
    {'number': 1528, 'title': 'Add CDP endpoint verification with exponential backoff for managed browsers', 'body': '## Summary\r\n\r\nManaged browsers sometimes failed to start...'},
    {'number': 1525, 'title': 'feat(mcp): migrate to modern HTTP transport with clean architecture refactor', 'body': '## 🚀 MCP Transport Modernization...'},
    {'number': 1513, 'title': 'task', 'body': '## Summary...'},
    {'number': 1316, 'title': '[Bug]: MCP SSE is failing with "Unexpected message"', 'body': '### crawl4ai version\n\n0.7.0...'},
    {'number': 1032, 'title': "[Bug]: Docker's playground can't set LLMConfig", 'body': "### crawl4ai version\n\n0.6.0 with local build docker..."},
    {'number': 964, 'title': '[Bug]: Running an ARM64 (aarch64) architecture Docker image on an AMD64 (x86_64) architecture machine, resulting in incompatibility', 'body': '### crawl4ai version\n\n0.5.0...'},
    {'number': 928, 'title': '[Bug]: docker hub images are outdated', 'body': '### crawl4ai version\n\n0.5.0...'},
    {'number': 460, 'title': 'Bug with "rotating proxies"', 'body': 'Rotating proxy is a very good feature...'},
    {'number': 453, 'title': 'Version 0.4.247: Persistent `fit_markdown` Issue', 'body': 'Based on the Usage Example...'},
    {'number': 413, 'title': 'Feature Request: Implement Question-Based Crawler with Search Engine Integration', 'body': 'Hi,\r\nI\'d like to contribute...'},
    {'number': 387, 'title': 'fix(browser)!: default to Chromium channel for new headless mode', 'body': 'BREAKING CHANGE: Updated `chrome_channel` to **`chromium`**...'},
    {'number': 382, 'title': 'Something wrong with the Playwright', 'body': 'The function of crawling relies on Playwright...'},
    {'number': 367, 'title': 'Unable to run headed browser in docker', 'body': 'When I set `headless=False`...'},
    {'number': 352, 'title': '为什么现在需要chrome？', 'body': '我在arm平台上使用该项目的基础能力...'},
    {'number': 26, 'title': 'Add some textual context to images, videos and audio in result.media, so that it can be useful in RAG', 'body': '## Issue...'},
    {'number': 7, 'title': 'problem running quickstart file', 'body': 'In line 86...'}
]

# ===================================================================
# FASE 2: ANÁLISE PROFUNDA (EXECUÇÃO REAL E IMPARCIAL)
# ===================================================================
print(f"\n\n--- FASE 2: Iniciando análise profunda e imparcial de {len(candidate_issues_data)} issues ---")
deep_analyses = {}

# O loop abaixo agora será executado, realizando a análise real.
for issue in candidate_issues_data:
    issue_number = issue.get("number")
    issue_title = issue.get("title", "Sem Título")
    issue_body = issue.get("body", "")

    if not issue_body:
        print(f"--- Ignorando Issue #{issue_number}: Corpo vazio. ---")
        continue
    
    print(f"\nAnalisando profundamente a Issue #{issue_number}: {issue_title}")

    prompt_deep_analysis = f"""
    Aja como um Arquiteto de Software Sênior. Analise a issue do GitHub abaixo.
    Sua tarefa é identificar o principal padrão de projeto (GoF) ou conceito arquitetural (ex: Concorrência, Cache, Proxy, Microsserviços, API Gateway, Strategy) que está sendo discutido, implementado ou solicitado.

    Se a issue for sobre um bug, qual conceito arquitetural o bug revela? (ex: um 'race condition' revela um problema de Concorrência).
    Se for uma nova feature, qual padrão ou conceito ela introduz?

    Título da Issue: "{issue_title}"
    Corpo da Issue:
    ---
    {issue_body}
    ---

    Responda em português e use o formato exato abaixo. Seja conciso e direto.

    PADRAO_IDENTIFICADO: [Nome do Padrão ou Conceito, ou "Nenhum Relevante" se for apenas um bug simples ou documentação]
    EVIDENCIA: [Cite a frase mais importante da issue que suporta sua conclusão e explique seu raciocínio em uma ou duas frases.]
    """
    
    try:
        analysis_result = ask_codellama(prompt_deep_analysis, max_new_tokens=1024)
        display(Markdown(f"**Análise da IA para a Issue #{issue_number}:**\n```\n{analysis_result}\n```"))
        deep_analyses[issue_number] = analysis_result
    except Exception as e:
        print(f"  -> Erro na análise profunda da issue #{issue_number}: {e}")
    
    # Limpando memória após cada chamada para evitar acúmulo
    gc.collect()
    torch.cuda.empty_cache()

# ===================================================================
# FASE 3: SÍNTESE EM MÚLTIPLAS PASSADAS (À PROVA DE FALHA DE MEMÓRIA)
# ===================================================================
# (Esta seção permanece a mesma do script anterior, pois já está otimizada)
print("\n\n--- FASE 3: SÍNTESE ARQUITETURAL (ESTRATÉGIA DIVIDIR-PARA-CONQUISTAR) ---")

if deep_analyses:
    # --- Fase 3.1: Agrupamento Temático ---
    print("\n--- Fase 3.1: Agrupando issues por tema arquitetural ---")
    
    light_context = ""
    for issue_num, analysis_text in deep_analyses.items():
        pattern_match = re.search(r"PADRAO_IDENTIFICADO:\s*(.*)", analysis_text)
        if pattern_match:
            pattern = pattern_match.group(1).strip()
            title = next((item['title'] for item in candidate_issues_data if item['number'] == issue_num), "N/A")
            light_context += f"Issue {issue_num}: {title} -> (Padrão: {pattern})\n"
            
    prompt_grouping = f"""
    Aja como um arquiteto de software. Abaixo está uma lista de issues e os padrões arquiteturais identificados.
    Sua tarefa é agrupar essas issues em 2 a 4 temas arquiteturais macro.

    Lista de Issues e Padrões:
    {light_context}

    Agrupe os NÚMEROS das issues em temas. Responda APENAS com um JSON. Exemplo:
    {{
      "Concorrência e Performance": [1572, ...],
      "Extensibilidade e Customização": [1527, 1180, ...],
      "Arquitetura de API e Serviços": [1550, 1519, ...]
    }}
    """
    
    themed_groups = {}
    try:
        grouping_result_str = ask_codellama(prompt_grouping, max_new_tokens=1024)
        json_str = grouping_result_str[grouping_result_str.find('{'):grouping_result_str.rfind('}')+1]
        themed_groups = json.loads(json_str)
        print("Agrupamento temático concluído:")
        display(themed_groups)
    except Exception as e:
        print(f"ERRO ao agrupar temas: {e}. Abortando síntese.")
        themed_groups = {}

    # --- Fase 3.2: Resumo por Tema ---
    if themed_groups:
        print("\n--- Fase 3.2: Gerando resumo para cada tema ---")
        theme_summaries = {}
        for theme, issue_nums in themed_groups.items():
            print(f"\nResumindo o tema: '{theme}'")
            
            theme_context = ""
            for num in issue_nums:
                if num in deep_analyses:
                    theme_context += f"--- Análise da Issue #{num} ---\n{deep_analyses[num]}\n\n"
            
            if theme_context:
                prompt_theme_summary = f"""
                Você é um arquiteto de software. Com base nas análises abaixo, escreva um parágrafo conciso resumindo as conclusões sobre o tema: "{theme}".

                Contexto do Tema:
                {theme_context}
                
                Responda em português.
                """
                try:
                    summary = ask_codellama(prompt_theme_summary, max_new_tokens=1024)
                    theme_summaries[theme] = summary
                    display(Markdown(f"**Resumo para '{theme}':**\n{summary}"))
                except Exception as e:
                    print(f"ERRO ao resumir o tema '{theme}': {e}")
                
                gc.collect()
                torch.cuda.empty_cache()

    # --- Fase 3.3: Síntese Final ---
    if themed_groups and theme_summaries:
        print("\n--- Fase 3.3: Montando o relatório final a partir dos resumos temáticos ---")
        
        final_synthesis_context = ""
        for theme, summary in theme_summaries.items():
            final_synthesis_context += f"### {theme}\n\n{summary}\n\n"
            
        prompt_final_report = f"""
        Como Arquiteto de Software Chefe, junte os resumos temáticos abaixo em um relatório final coeso.
        Escreva um "Resumo Executivo" no início para dar uma visão geral da arquitetura do 'crawl4ai'.

        Resumos Temáticos:
        {final_synthesis_context}

        Formate sua resposta como o relatório de arquitetura final em português.
        """
        
        try:
            final_report = ask_codellama(prompt_final_report, max_new_tokens=2048)
            display(Markdown(f"# Relatório Arquitetural Final (Baseado em Análise Imparcial)\n\n{final_report}"))
        except Exception as e:
            print(f"ERRO ao gerar o relatório final: {e}")

else:
    print("Nenhuma análise profunda foi concluída. Não é possível gerar o relatório.")

print("\n--- Processo de Análise Imparcial e Otimizada Concluído! ---")

Limpando a memória da GPU antes de começar...
Memória limpa. Iniciando o processo de análise.


--- FASE 2: Iniciando análise profunda e imparcial de 55 issues ---

Analisando profundamente a Issue #1595: docs: Self-hosting guide with comprehensive monitoring documentation


**Análise da IA para a Issue #1595:**
```
PADRAO_IDENTIFICADO: Nenhum Relevante

EVIDENCIA: A issue discute a adição de documentação de monitoramento para a implantação do Docker. Isso não é um exemplo de um padrão ou conceito arquitetural, mas sim uma solicitação de recurso para melhorar a documentação.
```


Analisando profundamente a Issue #1550: feat: Setup UV workspace with microservices architecture


**Análise da IA para a Issue #1550:**
```
PADRAO_IDENTIFICADO: Microsserviços

EVIDENCIA: A issue descreve a configuração inicial de um workspace UV com arquitetura de microsserviços, o que suporta a ideia de que o projeto está sendo implementado com esse padrão arquitetônico.
```


Analisando profundamente a Issue #1572: Fix cdp setting with managed browser 


...


Resumindo o tema: 'Arquitetura de API e Serviços'


**Resumo para 'Arquitetura de API e Serviços':**
Com base nas análises da issue #1550 e #1519, pode-se concluir que o padrão ou conceito arquitetural identificado é "Microsserviços". Isso suporta a ideia de que o projeto está sendo implementado com esse padrão arquitetônico. Já a issue #1519 não apresenta evidências de um padrão ou conceito arquitetural, mas sim uma tarefa técnica para implementar uma funcionalidade. Portanto, o padrão ou conceito arquitetural identificado é "Nenhum Relevante".


--- Fase 3.3: Montando o relatório final a partir dos resumos temáticos ---


# Relatório Arquitetural Final (Baseado em Análise Imparcial)

Resumo Executivo:

O crawl4ai é um projeto de software que tem como objetivo principal realizar crawling de páginas web e extrair informações relevantes. A arquitetura do projeto é composta por vários componentes, incluindo uma API, serviços e uma camada de persistência de dados.

Concorrência e Performance:

A issue #1572 discute um problema de concorrência e apresenta um PR para corrigir o problema. Isso suporta a conclusão de que a issue é relacionada a um problema de concorrência e que o arquiteto de software deve ter em mente essa questão ao projetar e desenvolver software.

Extensibilidade e Customização:

A extensibilidade e customização são conceitos importantes na arquitetura de software, pois permitem que os desenvolvedores adicionem novas funcionalidades e personalizem a aplicação de acordo com suas necessidades específicas. A issue #1527 discute a implementação de uma nova funcionalidade, a "Release/v0.7.5", que inclui a atualização de segurança e a adição de novos hooks para o Docker, o que não é um exemplo de um padrão ou conceito arquitetural, mas sim uma solicitação de recurso para uma nova funcionalidade. Já a issue #1180 discute a adição de uma nova classe `CallbackURLFilter` para filtrar URLs personalizadas durante a criação de um crawler profundo, o que também não é um exemplo de um padrão ou conceito arquitetural, mas sim uma funcionalidade específica para uma aplicação de crawling.

Arquitetura de API e Serviços:

A issue #1550 discute o uso de microsserviços na implementação do projeto. Já a issue #1519 não apresenta evidências de um padrão ou conceito arquitetural, mas sim uma tarefa técnica para implementar uma funcionalidade. Portanto, o padrão ou conceito arquitetural identificado é "Nenhum Relevante".

Conclusão:

O crawl4ai é um projeto de software que tem como objetivo principal realizar crawling de páginas web e extrair informações relevantes. A arquitetura do projeto é composta por vários componentes, incluindo uma API, serviços e uma camada de persistência de dados. A issue #1572 discute um problema de concorrência e apresenta um PR para corrigir o problema. A extensibilidade e customização são conceitos importantes na arquitetura de software, pois permitem que os desenvolvedores adicionem novas funcionalidades e personalizem a aplicação de acordo com suas necessidades específicas. A issue #1527 discute a implementação de uma nova funcionalidade, a "Release/v0.7.5", que inclui a atualização de segurança e a adição de novos hooks para o Docker, o que não é um exemplo de um padrão ou conceito arquitetural, mas sim uma solicitação de recurso para uma nova funcionalidade. Já a issue #1180 discute a adição de uma nova classe `CallbackURLFilter` para filtrar URLs personalizadas durante a criação de um crawler profundo, o que também não é um exemplo de um padrão ou conceito arquitetural, mas sim uma funcionalidade específica para uma aplicação de crawling. A issue #1550 discute o uso de microsserviços na implementação do projeto. Já a issue #1519 não apresenta evidências de um padrão ou conceito arquitetural, mas sim uma tarefa técnica para implementar uma funcionalidade. Portanto, o padrão ou conceito arquitetural identificado é "Nenhum Relevante".


--- Processo de Análise Imparcial e Otimizada Concluído! ---
